In [1]:
import os
import pandas as pd 

In [198]:
def read_csv_file(filename=""):
    df_raw = pd.read_csv(filename)
    return df_raw

data_list = []
f_name = "../../../JupyterNotebooks/Comparison_Multivariate/dataset/DivvyBikes/raw/2021{}-divvy-tripdata.csv"
for i in ("01","02","03","04","05","06","08","09","10","11","12"):
#for i in ("08","09","10"):
    data_list.append(read_csv_file (f_name.format(i)))

In [199]:
df_raw = pd.concat(data_list)

In [200]:
df_raw['started_at'] = pd.to_datetime(df_raw['started_at'])
df_raw.set_index('started_at', inplace=True)


In [201]:
checkouts_per_hour = df_raw.groupby('start_station_id').resample('1H').size()
checkouts_per_hour = checkouts_per_hour.reset_index(name='number_of_checkouts')
pivot_df = checkouts_per_hour.pivot(index='started_at', columns='start_station_id', values='number_of_checkouts')



In [202]:
pivot_df.fillna(0).to_csv("../../../JupyterNotebooks/Comparison_Multivariate/dataset/DivvyBikes/raw/2021-12-months-divvy-hourly-tripdata.csv")

In [223]:
def sum_nextday_checkout(station_id):
    column_filled = pivot_df.fillna(0)[station_id]
    shifted_rolling_sums = column_filled.rolling(window=24, min_periods=1).sum().shift(-24)
    
    ## this has to start from 2, otherwise the last entry will have the sum of all the element in datasets
    for i in range(2, 25):
      shifted_rolling_sums.iloc[-i] = column_filled.iloc[-i+1:].sum() 
    shifted_rolling_sums = shifted_rolling_sums.fillna(0)
    return (shifted_rolling_sums)


In [228]:
for ic in pivot_df.columns:
    pivot_df[ic+"_nextday"] = sum_nextday_checkout(ic)

In [233]:
pivot_df.fillna(0).to_csv("DivvyBikes_hourly_TS_daily_data.csv")

In [235]:
!head DivvyBikes_hourly_TS_daily_data.csv

started_at,13001,13006,13008,13011,13016,13017,13021,13022,13028,13029,13033,13034,13036,13037,13042,13045,13050,13053,13056,13058,13059,13061,13063,13068,13071,13073,13074,13075,13080,13081,13083,13084,13085,13089,13091,13096,13099,13102,13108,13109,13124,13128,13132,13133,13134,13135,13136,13137,13138,13139,13143,13144,13146,13150,13154,13155,13156,13157,13158,13160,13162,13163,13164,13165,13179,13191,13192,13193,13194,13196,13197,13206,13213,13215,13216,13217,13221,13224,13229,13235,13241,13242,13243,13245,13247,13248,13249,13253,13256,13257,13258,13259,13263,13265,13266,13268,13269,13271,13276,13277,13278,13285,13288,13289,13290,13292,13294,13296,13300,13303,13304,13307,13319,13323,13325,13326,13331,13332,13338,13341,13345,13353,13354,13379,13389,13398,13409,13420,13427,13430,13431,13432,13434,15442,15443,15445,15446,15449,15470,15491,15529,15530,15534,15535,15539,15541,15542,15544,15545,15546,15550,15571,15575,15578,15585,15597,15599,15615,15621,15622,15623,15624,15628,15631,15632

### Code for daily data 
- Following code use the daily data for hourly timestamps, i.e. each of the hourly timestamp will have the number of checkouts for next 24 hours. This will help understand the need of a station for next 24 hours. 
- The model will capture the weekly, monthly seasonality. 


In [ ]:
mdata = pd.read_csv('dataset/DivvyBikes/df2021-8-9-10.csv')
hourly_checkout = [ic for ic in mdata.columns if "_next1daycheckout" in ic]
hourly_checkout.insert(0,"checkouthour") ## this will be replaced with date at the end
hourly_checkout=hourly_checkout[:-1]
mdata = mdata[hourly_checkout[:201]]
mdata.rename(columns={"checkouthour":"date"},inplace=True)
mdata.set_index('date', inplace = True)
mdata.rename(columns=lambda x: x.replace('_next1daycheckout', '_checkout'), inplace=True)
mdata.to_csv("dataset/DivvyBikes/df2021-8-9-10_VAR_nextdayAshrdata.csv")
mdata